
# BreastDCEDL 
## View data snd model  with nifti files
#### Author: Tomer Fridman
#### Date: 2025-02-22
> BreastDCEDL/SPY2/BreastDCEDL_modeling_with_nifti_files.ipynb

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/naomifridman/BreastDCEDL/blob/main/BreastDCEDL_modeling_with_nifti_files.ipynb)

```
BreastDCEDL/
├── DUKE/
│   ├── dce/
│   │   ├── Breast_MRI_001_duke_aqc_0.nii.gz
│   │   ├── Breast_MRI_001_duke_aqc_1.nii.gz
│   │   ├── Breast_MRI_001_duke_aqc_2.nii.gz
│   │   ├── Breast_MRI_001_duke_aqc_3.nii.gz
│   │   ├── Breast_MRI_001_duke_aqc_4.nii.gz
│   │   ├── Breast_MRI_002_duke_aqc_0.nii.gz
│   │   └── ...
│   └── mask/
│       ├── Breast_MRI_001_duke_mask.nii.gz
│       ├── Breast_MRI_002_duke_mask.nii.gz
│       └── ...
├── ISPY1/
│   ├── dce/
│   │   ├── ISPY1_1239_spy1_vis1_dce_aqc_0.nii.gz
│   │   ├── ISPY1_1239_spy1_vis1_dce_aqc_1.nii.gz
│   │   ├── ISPY1_1239_spy1_vis1_dce_aqc_2.nii.gz
│   │   ├── ISPY1_1239_spy1_vis1_dce_aqc_3.nii.gz
│   │   └── ...
│   └── mask/
│       ├── ISPY1_1239_spy1_vis1_mask.nii.gz
│       └── ...
├── ISPY2/
│   ├── dce/
│   │   ├── ISPY2-797427_spy2_vis1_dce_aqc_0.nii.gz
│   │   ├── ISPY2-797427_spy2_vis1_dce_aqc_1.nii.gz
│   │   ├── ...
│   │   ├── ISPY2-797427_spy2_vis1_dce_aqc_6.nii.gz
│   │   └── ...
│   └── mask/
│       ├── ACRIN-6698-103939_spy2_vis1_mask.nii.gz
│       └── ...
└── metadata/
    ├── BreastDCEDL_metadata.csv
    ├── BreastDCEDL_duke_metadata.csv
    ├── BreastDCEDL_spy1_metadata.csv
    └── BreastDCEDL_spy2_metadata.csv
```

In [90]:

from PIL import Image
import requests
import argparse
import datetime
import json
import numpy as np
import os
import time
from pathlib import Path
from sklearn.metrics import classification_report,auc,roc_auc_score,recall_score,precision_score
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [91]:
import os
import numpy as np
import pandas as pd
from PIL import Image

In [92]:
import warnings
warnings.filterwarnings('ignore', '.*do not.*', )
warnings.warn('Do not show this message')

import os
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [93]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [94]:
from glob import glob
#from skimage import io
from sklearn.utils import shuffle

#from nipype.interfaces.ants import N4BiasFieldCorrection
import sys
import os
import ast

In [95]:
import warnings
warnings.filterwarnings('ignore')

In [96]:

my_nifti_data =True
git_samples = False

In [97]:
# Check if running in Google Colab

if 'google.colab' in str(get_ipython()):
    print("Running in Google Colab")
    # Clone the repository
    !git clone https://github.com/naomifridman/BreastDCEDL.git

    # Change to the repository directory
    os.chdir('/content/BreastDCEDL')
    

In [98]:
# use if data nifti data is downloaded
if my_nifti_data:
    # Base path of the nifti data
    base_path="G:\\My Drive\\breast_mri"
    nifti_path= {'spy2':os.path.join(base_path,"SPY2","vis1","dce"),
                        'spy1':os.path.join(base_path,"SPY1","vis1","dce"),
                        'duke':os.path.join(base_path,"DUKE","min_dce")}

    mask_path={'spy2':os.path.join(base_path,"SPY2","vis1","mask"),
                    'spy1':os.path.join(base_path,"SPY1","vis1","mask"),
                    'duke':os.path.join(base_path,"DUKE","mask")}
    
    
# use for data_samples
if git_samples:
    base_path="."
    nifti_path= {'spy2':os.path.join('.',"ISPY2","data_samples","dce"),
                        'spy1':os.path.join('.',"ISPY1","data_samples","dce"),
                        'duke':os.path.join('.',"DUKE","data_samples","dce")}

    mask_path={'spy2':os.path.join('.',"ISPY2","data_samples","mask"),
                    'spy1':os.path.join('.',"ISPY1","data_samples","mask"),
                    'duke':os.path.join('.',"DUKE","data_samples","mask")}
    

In [99]:
base_path

'G:\\My Drive\\breast_mri'

In [100]:
import os, sys
sys.path.append(os.path.abspath(os.path.join('.',  'utils')))

import data_utils as ds

ds.setup_paths(base_path, nifti_path, mask_path)

{'spy2': 'G:\\My Drive\\breast_mri\\SPY2\\vis1\\dce', 'spy1': 'G:\\My Drive\\breast_mri\\SPY1\\vis1\\dce', 'duke': 'G:\\My Drive\\breast_mri\\DUKE\\min_dce'} {'spy2': 'G:\\My Drive\\breast_mri\\SPY2\\vis1\\mask', 'spy1': 'G:\\My Drive\\breast_mri\\SPY1\\vis1\\mask', 'duke': 'G:\\My Drive\\breast_mri\\DUKE\\mask'}


# Load metadata

In [101]:
df = pd.read_csv('BreastDCEDL_metadata.csv')
df.head()

,pid,pCR,n_xy,n_z,n_times,pre,post_early,post_late,slice_thick,xy_spacing,...,race_white,race_black,HR,HER2,HR_HER2_STATUS,TripleNeg,HER2pos,HRposHER2neg,dataset,test
0,ACRIN-6698-102212,0.0,256.0,80.0,8.0,0.0,2.0,6.0,2.0,0.605500,...,1.0,0.0,0.0,0.0,TripleNeg,1.0,0.0,0.0,spy2,1.0
1,ACRIN-6698-103939,0.0,256.0,64.0,7.0,0.0,2.0,5.0,2.5,0.585900,...,1.0,0.0,1.0,0.0,HRposHER2neg,0.0,0.0,1.0,spy2,0.0
2,ACRIN-6698-104268,0.0,256.0,80.0,8.0,0.0,2.0,6.0,2.0,0.703125,...,0.0,1.0,0.0,0.0,TripleNeg,1.0,0.0,0.0,spy2,0.0
3,ACRIN-6698-107700,0.0,256.0,80.0,8.0,0.0,2.0,6.0,2.0,0.664100,...,0.0,0.0,1.0,1.0,HER2pos,0.0,1.0,0.0,spy2,0.0
4,ACRIN-6698-108969,1.0,256.0,80.0,8.0,0.0,2.0,6.0,2.0,0.703125,...,1.0,0.0,0.0,1.0,HER2pos,0.0,1.0,0.0,spy2,0.0


In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2070 entries, 0 to 2069
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pid             2070 non-null   object 
 1   pCR             1452 non-null   float64
 2   n_xy            2070 non-null   float64
 3   n_z             2070 non-null   float64
 4   n_times         2070 non-null   float64
 5   pre             2070 non-null   float64
 6   post_early      2070 non-null   float64
 7   post_late       2070 non-null   float64
 8   slice_thick     2070 non-null   float64
 9   xy_spacing      2070 non-null   float64
 10  mask_start      2070 non-null   float64
 11  mask_end        2070 non-null   float64
 12  sraw            2070 non-null   float64
 13  eraw            2070 non-null   float64
 14  scol            2070 non-null   float64
 15  ecol            2070 non-null   float64
 16  tum_vol         1456 non-null   float64
 17  age             2070 non-null   f

In [103]:
df[~df.pCR.isna()].info()

<class 'pandas.core.frame.DataFrame'>
Index: 1452 entries, 0 to 1451
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pid             1452 non-null   object 
 1   pCR             1452 non-null   float64
 2   n_xy            1452 non-null   float64
 3   n_z             1452 non-null   float64
 4   n_times         1452 non-null   float64
 5   pre             1452 non-null   float64
 6   post_early      1452 non-null   float64
 7   post_late       1452 non-null   float64
 8   slice_thick     1452 non-null   float64
 9   xy_spacing      1452 non-null   float64
 10  mask_start      1452 non-null   float64
 11  mask_end        1452 non-null   float64
 12  sraw            1452 non-null   float64
 13  eraw            1452 non-null   float64
 14  scol            1452 non-null   float64
 15  ecol            1452 non-null   float64
 16  tum_vol         1452 non-null   float64
 17  age             1452 non-null   float6

In [104]:
df.dataset.value_counts()

dataset
spy2    982
duke    916
spy1    172
Name: count, dtype: int64

In [105]:
df[~df.pCR.isna()][df.dataset=='duke']

,pid,pCR,n_xy,n_z,n_times,pre,post_early,post_late,slice_thick,xy_spacing,...,race_white,race_black,HR,HER2,HR_HER2_STATUS,TripleNeg,HER2pos,HRposHER2neg,dataset,test
1154,Breast_MRI_001,0.0,448.0,160.0,5.0,0.0,1.0,4.0,1.1,0.803571,...,0.0,1.0,0.0,1.0,HER2pos,0.0,1.0,0.0,duke,0.0
1155,Breast_MRI_002,0.0,512.0,142.0,4.0,0.0,1.0,3.0,1.3,0.585900,...,0.0,1.0,0.0,0.0,TripleNeg,1.0,0.0,0.0,duke,1.0
1156,Breast_MRI_003,0.0,512.0,156.0,5.0,0.0,1.0,4.0,2.0,0.683600,...,1.0,0.0,1.0,0.0,HRposHER2neg,0.0,0.0,1.0,duke,0.0
1157,Breast_MRI_005,1.0,448.0,160.0,4.0,0.0,1.0,3.0,2.0,0.781250,...,0.0,0.0,1.0,1.0,HER2pos,0.0,1.0,0.0,duke,0.0
1158,Breast_MRI_009,1.0,512.0,156.0,4.0,0.0,1.0,3.0,1.1,0.664100,...,1.0,0.0,0.0,0.0,TripleNeg,1.0,0.0,0.0,duke,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,Breast_MRI_907,0.0,448.0,160.0,5.0,0.0,1.0,4.0,1.1,0.758929,...,0.0,0.0,0.0,1.0,HER2pos,0.0,1.0,0.0,duke,1.0
1448,Breast_MRI_914,0.0,512.0,154.0,4.0,0.0,1.0,3.0,2.0,0.664100,...,1.0,0.0,0.0,0.0,TripleNeg,1.0,0.0,0.0,duke,0.0
1449,Breast_MRI_915,1.0,512.0,160.0,4.0,0.0,1.0,3.0,2.0,0.683600,...,1.0,0.0,1.0,0.0,HRposHER2neg,0.0,0.0,1.0,duke,2.0
1450,Breast_MRI_916,0.0,512.0,158.0,4.0,0.0,1.0,3.0,1.3,0.664100,...,0.0,0.0,1.0,0.0,HRposHER2neg,0.0,0.0,1.0,duke,0.0


In [106]:
df[df.test==1][~df.pCR.isna()].dataset.value_counts()

dataset
spy2    99
duke    43
spy1    35
Name: count, dtype: int64

In [107]:
df[df.test!=1][~df.pCR.isna()].dataset.value_counts()

dataset
spy2    883
duke    255
spy1    137
Name: count, dtype: int64

In [108]:
df[df.pCR.isna()].dataset.value_counts()

dataset
duke    618
Name: count, dtype: int64

In [109]:
for i,row in df.iterrows():
        p=row['pid']
        mc1 = -row['mask_start']+row['mask_end']+1
        mc2=-row['sraw']+row['eraw']+1
        mc3=-row['scol']+row['ecol']+1
        
        df.at[i,'bbox_vol']=row['slice_thick']*row['xy_spacing']*row['xy_spacing']*(mc1*mc2*mc3)#/1000.
        df.at[i,'bbox_count']=(mc1*mc2*mc3)
        df.at[i,'voxel_vol']=row['slice_thick']*row['xy_spacing']*row['xy_spacing']/1000
        
        

Breast_MRI_668 nan 1.0 27421.875
Breast_MRI_669 nan 1.0 3711.36776032
Breast_MRI_671 nan nan 1365.97665445507
Breast_MRI_673 nan 1.0 6139.121035200001
Breast_MRI_674 nan 2.0 75366.59255999999
Breast_MRI_675 nan 3.0 69494.52210624
Breast_MRI_676 nan 2.0 4591.286323624001
Breast_MRI_678 nan 1.0 17861.666805
Breast_MRI_680 nan 2.0 5971.684075472325
Breast_MRI_682 nan 2.0 181125.0
Breast_MRI_683 nan 2.0 116092.65859695274
Breast_MRI_689 nan 2.0 28402.255364000004
Breast_MRI_690 nan 1.0 49605.26586525815
Breast_MRI_692 nan 1.0 16541.926649819998
Breast_MRI_695 nan 3.0 208044.43359375
Breast_MRI_696 nan 1.0 8294.00566037823
Breast_MRI_698 nan 1.0 4398.44453125
Breast_MRI_699 nan 1.0 11698.289171039998
Breast_MRI_701 nan 1.0 561792.102462976
Breast_MRI_702 nan 1.0 3421.27733490602
Breast_MRI_703 nan 2.0 195503.22015609004
Breast_MRI_704 nan 4.0 1010213.7789808798
Breast_MRI_706 nan 3.0 418356.33364978206
Breast_MRI_707 nan 1.0 6688.775829149704
Breast_MRI_708 nan 3.0 394877.7912063601
Breast_

# ML

## Predict on Metadata only without Deep Learning

In [110]:
df.shape

(2070, 32)

In [111]:
df=df[~df.pCR.isna()]
df.shape

(1452, 32)

In [112]:
df_pred = pd.read_csv('./transformer_models/BreastDCEDL_vit_pcr_predictions.csv')
df=df.merge(df_pred[['pid','DL_pred_pcr_mean','DL_pred_pcr_min']], on='pid', how='left')


In [113]:
pd.crosstab(df.test, df.pCR)

pCR,0.0,1.0
test,,
0.0,777,322
1.0,124,53
2.0,123,53


In [114]:
df['duke']=np.where(df.dataset=='duke',1,0)
df['spy']=np.where(df.dataset!='duke',1,0)
df['spy2']=np.where(df.dataset=='spy2',1,0)
df['box_to_tum']=df['bbox_vol']/df['tum_vol']

In [115]:
pd.crosstab(df[df.test==1].pCR,df[df.test==1].dataset)



dataset,duke,spy1,spy2
pCR,,,
0.0,34,23,67
1.0,9,12,32


In [116]:
pd.crosstab(df[df.test==1].pCR,df[df.test==1].HR_HER2_STATUS)

HR_HER2_STATUS,HER2pos,HRposHER2neg,TripleNeg
pCR,,,
0.0,25,58,40
1.0,18,11,23


# Predict

In [117]:
pcols=['duke','spy2',
          # 'box_to_tum',
           
        'HR', 'HER2',  
           'age',
        'race_white',
          # 'race_black',  
      
         'bbox_vol',
       'tum_vol',
        
       'DL_pred_pcr_mean',
       'DL_pred_pcr_min'
          ]

In [118]:
df_test=df[df.test==1]
df_valid=df[df.test==2]
df_train=df[df.test==0]
df_val_train=df[df.test!=1]

In [119]:
    print('==== All')
    X_train = df_train[pcols].values
    y_train = df_train['pCR'].values

    X_test = df_test[pcols].values
    y_test=df_test.pCR.values
    X_test.shape,y_test
    
    # Standardize features
    scaler = StandardScaler()
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 1. Random Forest
    rf_model = RandomForestClassifier(
        n_estimators=600,
        max_depth=None,
        min_samples_split=2,
        random_state=42,
        #class_weight='balanced'
    )

    # Train Random Forest
    rf_model.fit(X_train_scaled, y_train)

    # Make predictions with Random Forest
    rf_pred = rf_model.predict(X_test_scaled)
    rf_pred_proba = rf_model.predict_proba(X_test_scaled)[:, 1]

    # Calculate metrics for Random Forest
    rf_auc = roc_auc_score(y_test, rf_pred_proba)
    rf_f1 = f1_score(y_test, rf_pred)
    rf_accuracy = accuracy_score(y_test, rf_pred)

    # y, y_pred, tlt = 'Train',classes=['Negative', 'pCR pos'],
    #            c_rep = 0, Thresh=0.5, plot_roc_c=0, plot_cm=0):
    ds.report_full(y=y_test, y_pred=rf_pred_proba, tlt='Test',
                plot_roc_c=0)
    # Print Random Forest results
    print("Random Forest Results:")
    print(f"AUC: {rf_auc:.4f}")
    print(f"Accuracy: {rf_accuracy:.4f}")
    print(f"F1-score: {rf_f1:.4f}")
    
    

==== All

 Test  Accuracy:  0.74  AUC:  0.724   Specificity: 0.935   Sensitivity: 0.283 NPV: 0.753 Precision: 0.652
    No  Yes
0  116    8
1   38   15
Random Forest Results:
AUC: 0.7242
Accuracy: 0.7401
F1-score: 0.3947


# by dataset

In [120]:
pcols=[#'duke','spy2',
          # 'box_to_tum',
           
        'HR', 'HER2',  
           'age',
        'race_white',
        #   'race_black',  
      
         'bbox_vol',
       'tum_vol',
        
       'DL_pred_pcr_mean',
       'DL_pred_pcr_min'
          ]

In [121]:
    print('==== spy1')
    X_train = df_train[pcols].values
    y_train = df_train['pCR'].values

    X_test = df_test[pcols][df_test.dataset=='spy1'].values
    y_test=df_test[df_test.dataset=='spy1'].pCR.values
    X_test.shape,y_test
    
    # Standardize features
    scaler = StandardScaler()
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 1. Random Forest
    rf_model = RandomForestClassifier(
        n_estimators=600,
        max_depth=None,
        min_samples_split=2,
        random_state=42,
        #class_weight='balanced'
    )

    # Train Random Forest
    rf_model.fit(X_train_scaled, y_train)

    # Make predictions with Random Forest
    rf_pred = rf_model.predict(X_test_scaled)
    rf_pred_proba = rf_model.predict_proba(X_test_scaled)[:, 1]

    # Calculate metrics for Random Forest
    rf_auc = roc_auc_score(y_test, rf_pred_proba)
    rf_f1 = f1_score(y_test, rf_pred)
    rf_accuracy = accuracy_score(y_test, rf_pred)

    # y, y_pred, tlt = 'Train',classes=['Negative', 'pCR pos'],
    #            c_rep = 0, Thresh=0.5, plot_roc_c=0, plot_cm=0):
    ds.report_full(y=y_test, y_pred=rf_pred_proba, tlt='Test',
                plot_roc_c=0)
    # Print Random Forest results
    print("Random Forest Results:")
    print(f"AUC: {rf_auc:.4f}")
    print(f"Accuracy: {rf_accuracy:.4f}")
    print(f"F1-score: {rf_f1:.4f}")
    
    

==== spy1

 Test  Accuracy:  0.743  AUC:  0.685   Specificity: 1.0   Sensitivity: 0.25 NPV: 0.719 Precision: 1.0
   No  Yes
0  23    0
1   9    3
Random Forest Results:
AUC: 0.6848
Accuracy: 0.7429
F1-score: 0.4000


In [122]:
    print('=== duke')
    X_train = df_train[pcols].values
    y_train = df_train['pCR'].values

    X_test = df_test[pcols][df_test.dataset=='duke'].values
    y_test=df_test[df_test.dataset=='duke'].pCR.values
    print(X_test.shape,y_test.sum())
    
    # Standardize features
    scaler = StandardScaler()
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 1. Random Forest
    rf_model = RandomForestClassifier(
        n_estimators=600,
        max_depth=None,
        min_samples_split=2,
        random_state=42,
    )

    # Train Random Forest
    rf_model.fit(X_train_scaled, y_train)

    # Make predictions with Random Forest
    rf_pred = rf_model.predict(X_test_scaled)
    rf_pred_proba = rf_model.predict_proba(X_test_scaled)[:, 1]

    # Calculate metrics for Random Forest
    rf_auc = roc_auc_score(y_test, rf_pred_proba)
    rf_f1 = f1_score(y_test, rf_pred)
    rf_accuracy = accuracy_score(y_test, rf_pred)

    # y, y_pred, tlt = 'Train',classes=['Negative', 'pCR pos'],
    #            c_rep = 0, Thresh=0.5, plot_roc_c=0, plot_cm=0):
    ds.report_full(y=y_test, y_pred=rf_pred_proba, tlt='Test',
                plot_roc_c=0)
    # Print Random Forest results
    print("Random Forest Results:")
    print(f"AUC: {rf_auc:.4f}")
    print(f"Accuracy: {rf_accuracy:.4f}")
    print(f"F1-score: {rf_f1:.4f}")

=== duke
(43, 8) 9.0

 Test  Accuracy:  0.698  AUC:  0.564   Specificity: 0.824   Sensitivity: 0.222 NPV: 0.8 Precision: 0.25
   No  Yes
0  28    6
1   7    2
Random Forest Results:
AUC: 0.5637
Accuracy: 0.6977
F1-score: 0.2353


In [123]:
    print('spy2')
    X_train = df_train[pcols].values
    y_train = df_train['pCR'].values

    X_test = df_test[pcols][df_test.dataset=='spy2'].values
    y_test=df_test[df_test.dataset=='spy2'].pCR.values
    X_test.shape,y_test
    
    # Standardize features
    scaler = StandardScaler()
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 1. Random Forest
    rf_model = RandomForestClassifier(
        n_estimators=600,
        max_depth=None,
        min_samples_split=2,
        random_state=42,
        #class_weight='balanced'
    )

    # Train Random Forest
    rf_model.fit(X_train_scaled, y_train)

    # Make predictions with Random Forest
    rf_pred = rf_model.predict(X_test_scaled)
    rf_pred_proba = rf_model.predict_proba(X_test_scaled)[:, 1]

    # Calculate metrics for Random Forest
    rf_auc = roc_auc_score(y_test, rf_pred_proba)
    rf_f1 = f1_score(y_test, rf_pred)
    rf_accuracy = accuracy_score(y_test, rf_pred)

    # y, y_pred, tlt = 'Train',classes=['Negative', 'pCR pos'],
    #            c_rep = 0, Thresh=0.5, plot_roc_c=0, plot_cm=0):
    ds.report_full(y=y_test, y_pred=rf_pred_proba, tlt='Test',
                plot_roc_c=0)
    # Print Random Forest results
    print("Random Forest Results:")
    print(f"AUC: {rf_auc:.4f}")
    print(f"Accuracy: {rf_accuracy:.4f}")
    print(f"F1-score: {rf_f1:.4f}")

spy2

 Test  Accuracy:  0.697  AUC:  0.774   Specificity: 0.94   Sensitivity: 0.188 NPV: 0.708 Precision: 0.6
   No  Yes
0  63    4
1  26    6
Random Forest Results:
AUC: 0.7745
Accuracy: 0.6970
F1-score: 0.2857


In [124]:
pcols=['duke','spy2',
          # 'box_to_tum',
    #    'HR', 'HER2',  
       
           'age',
        'race_white',
        #   'race_black',  
      
         'bbox_vol',
       'tum_vol', #'HR', 'HER2',#'TripleNeg',
        
       'DL_pred_pcr_mean',
       'DL_pred_pcr_min'
          ]

In [125]:
df_train=df[df.test==0]
df_val=df[df.test==2]
df_test=df[df.test==1]
df_val_train=df[df.test!=1]

In [126]:
    print('===HRposHER2neg')
    X_train = df_train[pcols].values
    y_train = df_train['pCR'].values

    X_val = df_val[pcols][df_val.HRposHER2neg==1].values
    y_val=df_val[df_val.HRposHER2neg==1].pCR.values
    
    X_val_train = df_val_train[pcols][df_val_train.HRposHER2neg==1].values
    y_val_train=df_val_train[df_val_train.HRposHER2neg==1].pCR.values
    
    X_test = df_test[pcols][df_test.HRposHER2neg==1].values
    y_test=df_test[df_test.HRposHER2neg==1].pCR.values
    
    
    # Standardize features
    scaler = StandardScaler()
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_val_scaled = scaler.transform(X_val)
    X_val_train_scaled = scaler.transform(X_val_train)

    # 1. Random Forest
    rf_model = RandomForestClassifier(
        n_estimators=600,
        max_depth=None,
        min_samples_split=2,
        random_state=42,
        #class_weight='balanced'
    )

    # Train Random Forest
    rf_model.fit(X_train_scaled, y_train)

    # Make predictions with Random Forest
    rf_pred = rf_model.predict(X_test_scaled)
    rf_pred_proba = rf_model.predict_proba(X_test_scaled)[:, 1]

    # Calculate metrics for Random Forest
    rf_auc = roc_auc_score(y_test, rf_pred_proba)
    rf_f1 = f1_score(y_test, rf_pred)
    rf_accuracy = accuracy_score(y_test, rf_pred)

    
    ds.report_full(y=y_test, y_pred=rf_pred_proba, tlt='Test',
                plot_roc_c=0)
    
    
    

===HRposHER2neg

 Test  Accuracy:  0.899  AUC:  0.734   Specificity: 1.0   Sensitivity: 0.364 NPV: 0.892 Precision: 1.0
   No  Yes
0  58    0
1   7    4


In [127]:
    print('===TripleNeg')
    X_train = df_train[pcols].values
    y_train = df_train['pCR'].values

    X_val = df_val[pcols][df_val.TripleNeg==1].values
    y_val=df_val[df_val.TripleNeg==1].pCR.values
    
    X_test = df_test[pcols][df_test.TripleNeg==1].values
    y_test=df_test[df_test.TripleNeg==1].pCR.values
    
    
    # Standardize features
    scaler = StandardScaler()
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_val_scaled = scaler.transform(X_val)
    X_val_train_scaled = scaler.transform(X_val_train)

    # 1. Random Forest
    rf_model = RandomForestClassifier(
        n_estimators=400,
        max_depth=None,
        min_samples_split=2,
        random_state=42,
    )

    # Train Random Forest
    rf_model.fit(X_train_scaled, y_train)

    # Make predictions with Random Forest
    rf_pred = rf_model.predict(X_test_scaled)
    rf_pred_proba = rf_model.predict_proba(X_test_scaled)[:, 1]

    # Calculate metrics for Random Forest
    rf_auc = roc_auc_score(y_test, rf_pred_proba)
    rf_f1 = f1_score(y_test, rf_pred)
    rf_accuracy = accuracy_score(y_test, rf_pred)

    
    ds.report_full(y=y_test, y_pred=rf_pred_proba, tlt='Test',
                plot_roc_c=0)
    # Print Random Forest results
    print("Random Forest Results:")
    print(f"AUC: {rf_auc:.4f}")
    print(f"Accuracy: {rf_accuracy:.4f}")
    print(f"F1-score: {rf_f1:.4f}")
    

===TripleNeg

 Test  Accuracy:  0.619  AUC:  0.533   Specificity: 0.975   Sensitivity: 0.0 NPV: 0.629 Precision: 0.0
   No  Yes
0  39    1
1  23    0
Random Forest Results:
AUC: 0.5326
Accuracy: 0.6190
F1-score: 0.0000


In [128]:
pcols=['duke','spy2',
          # 'box_to_tum',
       'HR', 
       #'HER2',  
       
           'age',
        'race_white',
          # 'race_black',  
      
         'bbox_vol',
       'tum_vol', #'HR', 'HER2',#'TripleNeg',
        
       'DL_pred_pcr_mean',
       'DL_pred_pcr_min'
          ]

In [129]:
    print('===HERPos')
    X_train = df_train[pcols].values
    y_train = df_train['pCR'].values

    X_val = df_val[pcols][df_val.HER2pos==1].values
    y_val=df_val[df_val.HER2pos==1].pCR.values
    
    X_val_train = df_val_train[pcols][df_val_train.HER2pos==1].values
    y_val_train=df_val_train[df_val_train.HER2pos==1].pCR.values
    
    X_test = df_test[pcols][df_test.HER2pos==1].values
    y_test=df_test[df_test.HER2pos==1].pCR.values
    
    
    # Standardize features
    scaler = StandardScaler()
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_val_scaled = scaler.transform(X_val)
    X_val_train_scaled = scaler.transform(X_val_train)

    # 1. Random Forest
    rf_model = RandomForestClassifier(
        n_estimators=600,
        max_depth=None,
        min_samples_split=2,
        random_state=42,
        #class_weight='balanced'
    )

    # Train Random Forest
    rf_model.fit(X_train_scaled, y_train)

    # Make predictions with Random Forest
    rf_pred = rf_model.predict(X_test_scaled)
    rf_pred_proba = rf_model.predict_proba(X_test_scaled)[:, 1]

    # Calculate metrics for Random Forest
    rf_auc = roc_auc_score(y_test, rf_pred_proba)
    rf_f1 = f1_score(y_test, rf_pred)
    rf_accuracy = accuracy_score(y_test, rf_pred)

    
    ds.report_full(y=y_test, y_pred=rf_pred_proba, tlt='Test',
                plot_roc_c=0)
    # Print Random Forest results
    print("Random Forest Results:")
    print(f"AUC: {rf_auc:.4f}")
    print(f"Accuracy: {rf_accuracy:.4f}")
    print(f"F1-score: {rf_f1:.4f}")
    

===HERPos

 Test  Accuracy:  0.674  AUC:  0.65   Specificity: 1.0   Sensitivity: 0.222 NPV: 0.641 Precision: 1.0
   No  Yes
0  25    0
1  14    4
Random Forest Results:
AUC: 0.6500
Accuracy: 0.6744
F1-score: 0.3636
